In [ ]:
!unzip /content/drive/MyDrive/superAI_lv2/hackathon/Final_5hacks/nithan_chadok/chadok-hybrid-ocr-ner.zip

In [ ]:
! pip install -q pytesseract

In [ ]:
! sudo apt-get install tesseract-ocr-tha
! sudo tesseract --list-langs

# process

In [ ]:
from transformers import CamembertTokenizerFast, CamembertForTokenClassification, pipeline
# from transformers import RobertaTokenizerFast, RobertaForTokenClassification, pipeline

# tokenizer = CamembertTokenizerFast.from_pretrained("thanaphatt1/WangchanBERTa-LST20")
# model = CamembertForTokenClassification.from_pretrained("thanaphatt1/WangchanBERTa-LST20")
tokenizer = RobertaTokenizerFast.from_pretrained("lst-nectec/HoogBERTa-NER-lst20")
model = RobertaForTokenClassification.from_pretrained("lst-nectec/HoogBERTa-NER-lst20")

nlp = pipeline('token-classification', model=model, tokenizer=tokenizer, aggregation_strategy="none", ignore_labels=[], device=0)

In [ ]:
from PIL import Image
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os
import pandas as pd
import math
import pytesseract
from tqdm import tqdm

In [ ]:
tag_list = pd.read_csv('tag_list.csv')
tags = {row['tag']:row['class'] for _, row in tag_list.iterrows()}
kernel = np.array([
    [0, 0, 0],
    [0, 1, 0],
    [0,0,0]
])
preds = []

In [ ]:
from PIL import Image
image = Image.open('images/images/00001.jpg')
image

In [ ]:
output_tesseract = pytesseract.image_to_string(image, lang='tha')
print(output_tesseract)

In [ ]:
def getSkewAngle(cvImage) -> float:
    # Prep image, copy, convert to gray scale, blur, and threshold
    newImage = np.array(cvImage)
    gray = cv2.cvtColor(newImage, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (9, 9), 0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    # Apply dilate to merge text into meaningful lines/paragraphs.
    # Use larger kernel on X axis to merge characters into single line, cancelling out any spaces.
    # But use smaller kernel on Y axis to separate between different blocks of text
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (30, 5))
    dilate = cv2.dilate(thresh, kernel, iterations=2)

    # Find all contours
    contours, hierarchy = cv2.findContours(dilate, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key = cv2.contourArea, reverse = True)
    firstbox = []
    lastbox = []
    box = []
    for c in contours:
        rect = cv2.boundingRect(c)
        x,y,w,h = rect
        box.append([x,y,x+w,y+h])
        cv2.rectangle(newImage,(x,y),(x+w,y+h),(0,255,0),2)

    s_box = sorted(box, key = lambda x: x[1])
    f_box = s_box[0]
    l_box = s_box[-1]
    if -f_box[0]+l_box[0] == 0:
        angle = 0
    else:
        m = (-l_box[1]+f_box[1])/(-f_box[0]+l_box[0])
        degree = math.atan(m)
        angle = degree * 180 / math.pi

    # Find largest contour and surround in min area box
    largestContour = contours[0]
    minAreaRect = cv2.minAreaRect(largestContour)
#     cv2.imwrite("boxes.jpg", newImage)
    if angle <0:
        angle += 90
    elif angle > 0:
        angle -= 90

    return angle

In [ ]:
getSkewAngle(image)

In [ ]:
# Rotate the image around its center
def rotateImage(cvImage, angle: float):
    newImage = np.array(cvImage)
    (h, w) = newImage.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    newImage = cv2.warpAffine(newImage, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return newImage
def deskew(cvImage):
    angle = getSkewAngle(cvImage)
    return rotateImage(cvImage, -1.0 * angle)

In [ ]:
plt.imshow(deskew(image))

In [ ]:
# image = Image.open('/kaggle/input/nithan-chadok-hybrid-ocr-ner/images/images/00000.jpg')
inputcopy = deskew(image)
grayinput = cv2.cvtColor(inputcopy, cv2.COLOR_BGR2GRAY)
_, binaryImage = cv2.threshold(grayinput, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
kernelSize = (5, 5)
opIterations = 2
morphKernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernelSize)
dilateImage = cv2.morphologyEx(
    binaryImage,
    cv2.MORPH_DILATE,
    morphKernel,
    None,
    None,
    opIterations,
    cv2.BORDER_REFLECT101
)

contours, _ = cv2.findContours(dilateImage, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
mean_row = np.array([c[:, 0].mean(0)[1] for c in contours])
min_row = np.array([c[:, 0].min(0)[1] for c in contours], dtype='int')
max_row = np.array([c[:, 0].max(0)[1] for c in contours], dtype='int')
min_col = np.array([c[:, 0].min(0)[0] for c in contours], dtype='int')
max_col = np.array([c[:, 0].max(0)[0] for c in contours], dtype='int')
row_idx = np.argsort(mean_row)
words = []

for idx in range(len(row_idx)):
    xx = cv2.drawContours(np.zeros_like(image, dtype='uint8'), contours, row_idx[idx], (1, 1, 1), thickness=-1)
    im = (1 - inputcopy) * xx
    im = im[min_row[row_idx[idx]]:max_row[row_idx[idx]], min_col[row_idx[idx]]:max_col[row_idx[idx]]]
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    sharpened = cv2.filter2D(1 - gray, -1, kernel)

    thresh1 = (gray < 100).astype('uint8') * 255
    thresh2 = cv2.adaptiveThreshold(sharpened, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 21, 10)
    padded = cv2.copyMakeBorder(thresh1.copy(), 10, 10, 10, 10, cv2.BORDER_CONSTANT,value=(255,))
    word = pytesseract.image_to_string(padded, lang='tha').replace("_","[!und:]").replace(" ","[!und:]").replace('\x0c', '').replace('\n', '')
    print(word)
    if len(word) == 0:
        word = '[!und:]'
    words.append(word)
    preds.append(nlp(word)[0]['entity'])
words, preds

## read each word

In [ ]:
import pytesseract

def crop_image(image):
    grayinput = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binaryImage = cv2.threshold(grayinput, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    kernelSize = (5, 5)
    opIterations = 2
    morphKernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernelSize)
    dilateImage = cv2.morphologyEx(
        binaryImage,
        cv2.MORPH_DILATE,
        morphKernel,
        None,
        None,
        opIterations,
        cv2.BORDER_REFLECT101
        )
    contours, _ = cv2.findContours(dilateImage, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    mean_row = np.array([c[:, 0].mean(0)[1] for c in contours])
    min_row = np.array([c[:, 0].min(0)[1] for c in contours], dtype='int')
    max_row = np.array([c[:, 0].max(0)[1] for c in contours], dtype='int')
    min_col = np.array([c[:, 0].min(0)[0] for c in contours], dtype='int')
    max_col = np.array([c[:, 0].max(0)[0] for c in contours], dtype='int')
    row_idx = np.argsort(mean_row)
    return row_idx, contours, min_row, max_row, min_col, max_col

def image_to_words(row_idx, contours, min_row, max_row, min_col, max_col, image):
    words = []
    for idx in range(len(row_idx)):
        im = image[min_row[row_idx[idx]]:max_row[row_idx[idx]], min_col[row_idx[idx]]:max_col[row_idx[idx]]]
        word = pytesseract.image_to_string(im, lang='tha').replace("_","[!und:]").replace(" ","[!und:]").replace('\x0c', '').replace('\n', '')
        if len(word) == 0:
            word = '-'
        words.append(word)
    return words

In [ ]:
image = Image.open("images/images/00001.jpg")
image = np.array(image)
inputcopy = deskew(image)
row_idx, contours, min_row, max_row, min_col, max_col = crop_image(inputcopy)

num_rows = 2
num_cols = 7
fig, axes = plt.subplots(num_rows, num_cols, figsize=(10, 3))

for idx, ax in enumerate(axes.flat):
    if idx < len(row_idx):
        im = inputcopy[min_row[row_idx[idx]]:max_row[row_idx[idx]], min_col[row_idx[idx]]:max_col[row_idx[idx]]]
        ax.imshow(im, cmap='gray')
        ax.axis('off')
    else:
        ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
words = image_to_words(row_idx, contours, min_row, max_row, min_col, max_col, image)
print(words)

# submit

## transform & load data

In [ ]:
src = 'images/images'
txt = []

for i in tqdm(range(len(os.listdir(src)))):
    image = Image.open(f'{src}/{i:05d}.jpg')
    image = np.array(image)
    inputcopy = deskew(image)
    row_idx, contours, min_row, max_row, min_col, max_col = crop_image(inputcopy)
    words = image_to_words(row_idx, contours, min_row, max_row, min_col, max_col, inputcopy)
    txt.extend(words)

print(len(txt))

In [ ]:
df = pd.DataFrame({'Text': txt})
df.to_csv('text_data.csv', index=False)

## read test data

In [ ]:
NER_TAGS = [
       "O",
        "B_BRN",        "B_DES",        "B_DTM",        "B_LOC",        "B_MEA",        "B_NUM",        "B_ORG",        "B_PER",        "B_TRM",        "B_TTL",
       "I_BRN",        "I_DES",        "I_DTM",        "I_LOC",        "I_MEA",        "I_NUM",        "I_ORG",        "I_PER",        "I_TRM",        "I_TTL",
        "E_BRN",        "E_DES",        "E_DTM",        "E_LOC",        "E_MEA",        "E_NUM",        "E_ORG",        "E_PER",        "E_TRM",        "E_TTL"]

In [ ]:
txt = pd.read_csv('test.csv')
txt = txt['word']
txt

In [ ]:
text_list = []
for t in txt:
  text_list.append(" ".join(t))
text_list

## read model

In [ ]:
from transformers import RobertaTokenizerFast, RobertaForTokenClassification, pipeline

tokenizer = RobertaTokenizerFast.from_pretrained("lst-nectec/HoogBERTa-NER-lst20")
model = RobertaForTokenClassification.from_pretrained("lst-nectec/HoogBERTa-NER-lst20")
# Define the model and tokenizer
nlp = pipeline('token-classification', model=model, tokenizer=tokenizer, aggregation_strategy="none", ignore_labels=[])
nlp

## token & prediction

In [ ]:
preds = []
count = 0

raw_text = []

for text in tqdm(txt):
    # Split the text into words.
    text_follow_up = text.split()

    # Predict the text.
    pred = nlp(text)
    count += len(pred)
    # This below code is to handle the case that the text is not in the same order as the prediction.
    current = 0
    batch = []
    coming_text = ""

    while len(pred):
        # If the text is the same, we will append the entity.
        if pred[0]['word'].replace('</w>', '') == text_follow_up[current]:
            raw_text.append(pred[0]['word'])
            batch.append(pred[0]['entity'])
            pred.pop(0)
            current += 1
        else:
            # If the text is not the same, we will append the text to the coming_text.
            if coming_text + pred[0]['word'].replace('</w>', '') == text_follow_up[current]:
                raw_text.append(pred[0]['word'])
                batch.append(pred[0]['entity'])
                pred.pop(0)
                current += 1
                coming_text = ""
                continue

            coming_text += pred[0]['word']
            pred.pop(0)

    preds.extend(batch)

In [ ]:
pd.Series(preds).value_counts()

In [ ]:
tag_df = pd.read_csv('tag_list.csv')
tags = {row['tag']:row['class'] for _, row in tag_df.iterrows()}
tags

In [ ]:
submit = pd.read_csv('sample_submission.csv')

In [ ]:
submit.loc[3:, 'pred'] = preds[3:]
submit.loc[3:, 'pred'] = submit.pred.map(tags)
submit

In [ ]:
submit.pred = submit.pred.astype(int)
submit.pred.value_counts()

In [ ]:
submit.to_csv('nlp_testinfer_HoogBERTa_NER_lst20.csv', index=False)